In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.keys import Keys
import time

import os
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import PromptTemplate

from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [ ]:
# naver 검색
searchkey = "전라남도 광주 여행"
url = f"https://search.naver.com/search.naver?ssc=tab.blog.all&sm=tab_jum&query={searchkey}"

driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))
driver.implicitly_wait(10)
driver.get(url)
driver.implicitly_wait(10)

# 스크롤 아래로
actions = driver.find_element(By.CSS_SELECTOR, 'body')
actions.send_keys(Keys.END)

time.sleep(2)
actions.send_keys(Keys.END)
time.sleep(2)
actions.send_keys(Keys.END)
time.sleep(2)
actions.send_keys(Keys.END)
time.sleep(2)
actions.send_keys(Keys.END)
time.sleep(2)

In [ ]:
# 블로그 url 리스트화
title_element_list = driver.find_elements(By.CLASS_NAME,'title_link')

title_url_list =[]
for title_url in title_element_list:
    url_append = title_url.get_attribute('href')
    title_url_list.append(url_append)
driver.close()

print("크롤링 사이트 개수 : ", len(title_url_list))

In [ ]:
# api_key = os.getenv('OPENAI_API_KEY')

# 원하는 데이터 구조를 정의합니다.
class QAPair(BaseModel):
    question: str = Field(description="Question generated from the text")
    answer: str = Field(description="Answer related to the question")



prompt = PromptTemplate.from_template(
    """Context information is below. You are only aware of this context and nothing else.
---------------------

{context}

---------------------
Given this context, generate only questions based on the below query.
You are an Teacher/Professor in {domain}. 
Your task is to provide exactly **{num_questions}** question(s) for an upcoming quiz/examination. 
You are not to provide more or less than this number of questions. 
The question(s) should be diverse in nature across the document. 
The purpose of question(s) is to test the understanding of the students on the context information provided.
You must also provide the answer to each question. The answer should be based on the context information provided only.

Restrict the question(s) to the context information provided only.
QUESTION and ANSWER should be written in Korean. Response in JSON format which contains the `question` and `answer`.
ANSWER should be a complete sentence.

#Format:
```json
{{
    "QUESTION": "서울에서 전통 궁궐을 방문하려면 어떤 곳을 가야 합니까?",
    "ANSWER": "서울에서 전통 궁궐을 방문하려면 경복궁과 창덕궁을 가야 합니다."
}},
{{
    "QUESTION": "부산에서 유명한 해변과 시장의 이름은 무엇입니까?",
    "ANSWER": "부산에서 유명한 해변은 해운대 해변이고, 유명한 시장은 자갈치 시장입니다."
}},
{{
    "QUESTION": "제주도에서 볼 수 있는 주요 명소 세 곳은 어디입니까?",
    "ANSWER": "제주도에서 볼 수 있는 주요 명소는 성산 일출봉, 한라산, 천지연 폭포입니다."
}}
```
"""
)

In [ ]:
driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))
driver.implicitly_wait(10)
    
qa_dataset =[] # dataset 저장 리스트
# blog 정보 추출
for bs_url in title_url_list[:1]:
# bs_url = title_url_list [0]
    m_url = "https://m." + bs_url.replace("https://","") # mobile 버전으로 우회
    driver.get(m_url)
    driver.implicitly_wait(10)

    # 본문 내용
    contents_ele = driver.find_elements(By.CLASS_NAME,'se-main-container')
    try:
        contents_text = contents_ele[0].text

        # 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
        parser = JsonOutputParser(pydantic_object=QAPair)

        chain = (
            prompt
            | ChatOpenAI(
                model="gpt-4o",
                temperature=0,
                streaming=True,
                callbacks=[StreamingStdOutCallbackHandler()],
            )
            | parser
        )  # 체인을 구성합니다.

        qa_pair = []

        qa_pair = chain.invoke(
        {"context": contents_text, "domain": "여행", "num_questions": "10"}
        )
        qa_dataset.extend(qa_pair)
    except Exception as e:
        print(e)
    
driver.close()

In [ ]:
# json 저장
import json

with open("./DATA/전라남도_광주_여행_qa_pair.jsonl", "w", encoding="utf-8") as f:
    for qa in qa_dataset:
        qa_modified = {
            "instruction": qa["QUESTION"],
            "input": "",
            "output": qa["ANSWER"],
        }
        f.write(json.dumps(qa_modified, ensure_ascii=False) + "\n")

In [ ]:
# json load
from datasets import load_dataset

# JSONL 파일 경로
jsonl_file = "./DATA/qa_pair.jsonl"

# JSONL 파일을 Dataset으로 로드
dataset = load_dataset("json", data_files=jsonl_file)